In [1]:
#!/usr/bin/env python

In [2]:
import os
import sys
if os.path.exists('/home/chieh/code/wPlotLib'):
	sys.path.insert(0,'/home/chieh/code/wPlotLib')
if os.path.exists('/home/chieh/code/wuML'):
	sys.path.insert(0,'/home/chieh/code/wuML')

In [3]:
import wuml
import numpy as np
import torch
import wplotlib
from wuml import HSIC

In [4]:
data = wuml.wData(xpath='../../data/wine.csv', ypath='../../data/wine_label.csv', randomly_shuffle_batch=True,
					batch_size=20, label_type='discrete', encode_discrete_label_to_one_hot=True )

Remember that you are trying to "minimize" this objective

In [5]:
def costFunction(x, y, ŷ, ind):
	H = HSIC(ŷ, y, X_kernel='linear', Y_kernel='linear', sigma_type='mpd' )
	# compare the HSIC value against numpy library
	#y = y.detach().cpu().numpy()
	#ŷ = ŷ.detach().cpu().numpy()
	#H2 = HSIC(ŷ, y, X_kernel='linear', Y_kernel='linear', sigma_type='mpd' )
	#import pdb; pdb.set_trace()
	return -H  

----------------------------------------------<br>
Create basic network and train

In [6]:
bNet = wuml.basicNetwork(costFunction, data, networkStructure=[(30,'relu'),('bn', True), (50,'relu'),('bn', True),(3,'none')], max_epoch=500, learning_rate=0.001)
bNet.train(print_status=True)
Ŷ = bNet(data, output_type='ndarray')		#Takes Numpy array or Tensor as input and outputs a Tensor

Network Info:
	Learning rate: 0.001
	Max number of epochs: 500
	Cost Function: costFunction
	Train Loop Callback: None
	Device type: 'cuda'
	Network Structure
		Linear(in_features=13, out_features=30, bias=True) , relu
		BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) 
		Linear(in_features=30, out_features=50, bias=True) , relu
		BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) 
		Linear(in_features=50, out_features=3, bias=True) , none



In [7]:
Network_out_v_Label = np.hstack((Ŷ, data.Y))
wuml.jupyter_print(Network_out_v_Label)

,0,1,2,3,4,5
0,1.179747,-0.580604,-0.016019,1.0,0.0,0.0
1,1.576362,-0.484787,0.033831,1.0,0.0,0.0
2,2.084268,-0.653507,0.419791,1.0,0.0,0.0
3,2.880405,-1.346668,1.171079,1.0,0.0,0.0
4,-0.284758,0.186249,-0.591563,1.0,0.0,0.0
5,2.777499,-1.272872,1.077336,1.0,0.0,0.0
6,2.463265,-0.899210,0.707578,1.0,0.0,0.0
7,2.188525,-0.909954,0.611748,1.0,0.0,0.0
8,1.616215,-0.434696,0.029850,1.0,0.0,0.0
9,1.626730,-0.456946,0.041499,1.0,0.0,0.0
